In [2]:
import open3d as o3d
import numpy as np
import json
from glob import glob
import os
import cv2
from pathlib import Path

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Eye-in-Hand

## 1. Target PCD용 pcd 정합

### (1) 파일변환

In [15]:
MAIN_PATH = "/home/ros/llm_robot/data/Calibration/Eye-to-Hand4"

rgb_paths = glob(f"{MAIN_PATH}/color/*.jpg")
depth_paths = glob(f"{MAIN_PATH}/depth/*.npy")
intr_paths = glob(f"{MAIN_PATH}/intrinsics/*.json")
pose_paths = glob(f"{MAIN_PATH}/poses/*.json")
cam2ee_path = f"{MAIN_PATH}/cam2base.json"

In [16]:
depth_npy_dir = f"{MAIN_PATH}/depth"
output_dir = os.path.join(depth_npy_dir, "converted_png")

os.makedirs(output_dir, exist_ok=True)
depth_paths = glob(f"{depth_npy_dir}/*.npy")
converted_files = []
for npy_path in depth_paths:
    try:
        depth = np.load(npy_path).astype(np.uint16)
        filename = os.path.splitext(os.path.basename(npy_path))[0] + ".png"
        png_path = os.path.join(output_dir, filename)
        cv2.imwrite(png_path, depth)
        converted_files.append(png_path)
    except Exception as e:
        print(f"❌ {npy_path} 변환 실패: {e}")

converted_files[:5] 

['/home/ros/llm_robot/data/Calibration/Eye-to-Hand4/depth/converted_png/20_2025-08-07_19-18-58.png',
 '/home/ros/llm_robot/data/Calibration/Eye-to-Hand4/depth/converted_png/21_2025-08-07_19-19-24.png',
 '/home/ros/llm_robot/data/Calibration/Eye-to-Hand4/depth/converted_png/18_2025-08-07_19-18-00.png',
 '/home/ros/llm_robot/data/Calibration/Eye-to-Hand4/depth/converted_png/02_2025-08-07_19-14-05.png',
 '/home/ros/llm_robot/data/Calibration/Eye-to-Hand4/depth/converted_png/06_2025-08-07_19-14-48.png']

In [17]:
def rgbd_to_pcd(rgb_path, depth_path, intrinsic_path, depth_scale=1000.0, depth_trunc=1.5):
    color_raw = o3d.io.read_image(rgb_path)
    depth_raw = o3d.io.read_image(depth_path)

    with open(intrinsic_path, "r") as f:
        intr_data = json.load(f)["color_intrinsics"]
        
    width = intr_data["width"]
    height = intr_data["height"]
    fx = intr_data["fx"]
    fy = intr_data["fy"]
    cx = intr_data["ppx"]
    cy = intr_data["ppy"]
        
    intrinsic = o3d.camera.PinholeCameraIntrinsic(
        width=width, height=height, fx=fx, fy=fy, cx=cx, cy=cy
    )

    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color_raw, depth_raw,
        depth_scale=depth_scale,
        depth_trunc=depth_trunc,
        convert_rgb_to_intensity=False
    )
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic)
    return pcd

In [19]:
with open(intr_paths[0], "r") as f:
    intr_data = json.load(f)
    
intr_data["color_intrinsics"]["width"]

848

### (2) 여러 프레임을 T_cam2base로 정렬하여 병합

In [20]:
def merge_pcds_with_transforms(pcd_list, transform_list):
    merged = o3d.geometry.PointCloud()
    for pcd, T in zip(pcd_list, transform_list):
        pcd_transformed = pcd.transform(T)
        merged += pcd_transformed
    return merged

### (3) T_cam2base 계산

In [ ]:
with open(cam2ee_path, "r") as f:
    data = json.load(f)
R_cam2base = np.array(data["R_cam2base"])
t_cam2base = np.array(data["t_cam2base"]).reshape(3, 1)

T_cam2base = np.eye(4)
T_cam2base[:3, :3] = R_cam2base
T_cam2base[:3, 3] = t_cam2base.flatten()

In [12]:
def pose_to_transform(pose_file):
    with open(pose_file, "r") as f:
        pose = json.load(f)
    R_base2ee = np.array(pose["R_base2ee"])
    t_base2ee = np.array(pose["t_base2ee"]).reshape(3, 1)
    T_base2ee = np.eye(4)
    T_base2ee[:3, :3] = R_base2ee
    T_base2ee[:3, 3] = t_base2ee.flatten()
    return T_base2ee @ T_cam2ee  # T_cam2base = T_base2ee * T_cam2ee

In [13]:
pcds = []
T_list = []
for rgb, depth, intr, pose in zip(rgb_paths, converted_files, intr_paths, pose_paths):
    try:
        pcd = rgbd_to_pcd(rgb, depth, intr)
        T = pose_to_transform(pose)
        pcds.append(pcd)
        T_list.append(T)
    except Exception as e:
        print(f"❌ Failed to load {rgb}: {e}")

if pcds:
    try:
        merged_pcd = merge_pcds_with_transforms(pcds, T_list)
        save_path = "./merged_target_pcd.ply"
        o3d.io.write_point_cloud(save_path, merged_pcd)
        result = f"✅ 평균 기준 PCD 저장 완료: {save_path}"
    except Exception as e:
        result = f"❌ 평균 기준 PCD 저장 실패: {e}"
else:
    result = "❌ 변환 가능한 PCD가 없습니다."
    
result

❌ Failed to load /home/ros/llm_robot/data/Calibration/Eye-to-Hand2/color/2025-08-07_15-49-07.jpg: name 'T_cam2ee' is not defined


'❌ 변환 가능한 PCD가 없습니다.'

## 2. 관찰 PC 생성

In [22]:
pcd = rgbd_to_pcd(rgb_paths[0], converted_files[0], intr_paths[0])
o3d.io.write_point_cloud("./camera_observed.ply", pcd)

True

## 3. ICP 최적화 (Point-to-Point ICP)

In [ ]:
#### Eye in Hand #####
# === 초기 Tsai-Lenz 기반 ee2cam 변환행렬 불러오기 ===
T_ee2cam = np.eye(4)
T_ee2cam[:3, :3] = R_ee2cam  # Tsai-Lenz 결과
T_ee2cam[:3, 3] = t_ee2cam.flatten()

# === PCD 불러오기 ===
source_pcd = o3d.io.read_point_cloud("camera_observed.ply")  # EE 위치에서 관측한 것
target_pcd = o3d.io.read_point_cloud("merged_target_pcd.ply")  # 기준 Target

# === 초기 정렬 적용 ===
source_pcd.transform(T_ee2cam)

# === ICP 보정 ===
threshold = 0.01  # 1cm 허용 오차
icp_result = o3d.pipelines.registration.registration_icp(
    source_pcd, target_pcd,
    threshold,
    np.eye(4),
    o3d.pipelines.registration.TransformationEstimationPointToPoint()
)

# === 최종 보정 행렬 ===
T_icp = icp_result.transformation
T_ee2cam_refined = T_icp @ T_ee2cam

# === 최종 R, t 추출 ===
R_final = T_ee2cam_refined[:3, :3]
t_final = T_ee2cam_refined[:3, 3].reshape(3, 1)

In [33]:
result = {
    "R_cam2ee": R_final.tolist(),
    "t_cam2ee": t_final.flatten().tolist()  # 1D 리스트로 저장
}

# 저장
with open("optimized_cam2ee_icp_point.json", "w") as f:
    json.dump(result, f, indent=2)

## 4.Point-to-Plane ICP

In [35]:
# === 정규 벡터 추정 (PointToPlane ICP requires this)
source_pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.02, max_nn=30)
)
target_pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.02, max_nn=30)
)

# === ICP 실행 ===
threshold = 0.01  # 1cm
icp_result = o3d.pipelines.registration.registration_icp(
    source_pcd, target_pcd,
    threshold,
    np.eye(4),  # 이미 정렬했기 때문에 초기값은 identity
    o3d.pipelines.registration.TransformationEstimationPointToPlane()
)

# === 결과 확인 ===
print("✅ ICP fitness:", icp_result.fitness)
print("✅ ICP inlier RMSE:", icp_result.inlier_rmse)
print("✅ 보정 행렬 (delta T):")
print(icp_result.transformation)

✅ ICP fitness: 0.0
✅ ICP inlier RMSE: 0.0
✅ 보정 행렬 (delta T):
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [37]:
# === 최종 보정된 cam2ee = ΔT × 기존 T_cam2ee
T_refined = T_cam2ee @ icp_result.transformation 
R_final = T_refined[:3, :3]
t_final = T_refined[:3, 3].reshape(3, 1)

# === JSON 저장
result = {
    "R_cam2ee": R_final.tolist(),
    "t_cam2ee": t_final.flatten().tolist()
}
with open("optimized_cam2ee_icp_plane.json", "w") as f:
    json.dump(result, f, indent=2)
print("✅ 저장 완료: optimized_cam2ee_icp_plane.json")

✅ 저장 완료: optimized_cam2ee_icp_plane.json


### 번외. PCD 시각화

In [38]:
def visualize_ply(filepath):
    """
    단일 PLY 파일을 시각화하는 함수
    """
    pcd = o3d.io.read_point_cloud(filepath)
    print(f"Loaded point cloud: {pcd}")
    o3d.visualization.draw_geometries([pcd])

def animate_ply_sequence(directory, num_frames, basename="frame", ext=".ply", delay=0.1):
    """
    연속된 ply 파일 시퀀스를 애니메이션으로 시각화하는 함수.
    예: frame0000.ply, frame0001.ply, ...
    - directory: 파일 경로
    - num_frames: 총 프레임 수
    - basename: 파일명 기준 이름
    - ext: 확장자
    - delay: 프레임 간 대기 시간 (초)
    """
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    geometry = o3d.geometry.PointCloud()
    vis.add_geometry(geometry)

    for i in range(num_frames):
        filename = f"{directory}/{basename}{i:04d}{ext}"
        pcd = o3d.io.read_point_cloud(filename)
        geometry.points = pcd.points
        if pcd.has_colors():
            geometry.colors = pcd.colors
        vis.update_geometry(geometry)
        vis.poll_events()
        vis.update_renderer()
        o3d.utility.sleep(delay)

    vis.destroy_window()

In [40]:
visualize_ply("/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/merged_target_pcd.ply")

Loaded point cloud: PointCloud with 5462791 points.


# Eye-to-Hand

In [3]:
MAIN_PATH = "/home/ros/llm_robot/data/Calibration/Eye-to-Hand12"

rgb_paths = glob(f"{MAIN_PATH}/color/*.jpg")
depth_paths = glob(f"{MAIN_PATH}/depth/*.npy")
intr_paths = glob(f"{MAIN_PATH}/intrinsics/*.json")
pose_paths = glob(f"{MAIN_PATH}/poses/*.json")
charuco_files = sorted(Path(MAIN_PATH).glob("*.json"))[:25]

cam2ee_path = f"{MAIN_PATH}/cam2base.json"

In [4]:
depth_npy_dir = f"{MAIN_PATH}/depth"
output_dir = os.path.join(depth_npy_dir, "converted_png")

os.makedirs(output_dir, exist_ok=True)
depth_paths = glob(f"{depth_npy_dir}/*.npy")
converted_files = []
for npy_path in depth_paths:
    try:
        depth = np.load(npy_path).astype(np.uint16)
        filename = os.path.splitext(os.path.basename(npy_path))[0] + ".png"
        png_path = os.path.join(output_dir, filename)
        cv2.imwrite(png_path, depth)
        converted_files.append(png_path)
    except Exception as e:
        print(f"❌ {npy_path} 변환 실패: {e}")

converted_files[:5] 

['/home/ros/llm_robot/data/Calibration/Eye-to-Hand12/depth/converted_png/20_2025-08-27_12-19-31.png',
 '/home/ros/llm_robot/data/Calibration/Eye-to-Hand12/depth/converted_png/23_2025-08-27_12-20-04.png',
 '/home/ros/llm_robot/data/Calibration/Eye-to-Hand12/depth/converted_png/12_2025-08-27_12-17-44.png',
 '/home/ros/llm_robot/data/Calibration/Eye-to-Hand12/depth/converted_png/18_2025-08-27_12-19-15.png',
 '/home/ros/llm_robot/data/Calibration/Eye-to-Hand12/depth/converted_png/02_2025-08-27_12-15-58.png']

In [5]:
def rgbd_to_pcd(rgb_path, depth_path, intrinsic_path, depth_scale=1000.0, depth_trunc=1.5):
    color_raw = o3d.io.read_image(rgb_path)
    depth_raw = o3d.io.read_image(depth_path)
    # depth_raw = np.load(depth_path)

    with open(intrinsic_path, "r") as f:
        intr_data = json.load(f)["color_intrinsics"]
        
    width = intr_data["width"]
    height = intr_data["height"]
    fx = intr_data["fx"]
    fy = intr_data["fy"]
    cx = intr_data["ppx"]
    cy = intr_data["ppy"]
        
    intrinsic = o3d.camera.PinholeCameraIntrinsic(
        width=width, height=height, fx=fx, fy=fy, cx=cx, cy=cy
    )

    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color_raw, depth_raw,
        depth_scale=depth_scale,
        depth_trunc=depth_trunc,
        convert_rgb_to_intensity=False
    )
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic)
    return pcd

In [6]:
def merge_pcds_with_transforms(pcd_list, transform_list):
    merged = o3d.geometry.PointCloud()
    for pcd, T in zip(pcd_list, transform_list):
        pcd_transformed = pcd.transform(T)
        merged += pcd_transformed
    return merged

In [7]:
with open(cam2ee_path, "r") as f:
    data = json.load(f)
R_cam2base = np.array(data["R_cam2base"])
t_cam2base = np.array(data["t_cam2base"]).reshape(3, 1)

T_cam2base = np.eye(4)
T_cam2base[:3, :3] = R_cam2base
T_cam2base[:3, 3] = t_cam2base.flatten()

In [8]:
merged_pcd = o3d.geometry.PointCloud()
T_target_in_base =[]
for i, (pose_path, charuco_path) in enumerate(zip(pose_paths, charuco_files)):
    with open(pose_path, "r") as f1, open(charuco_path, "r") as f2:
        pose = json.load(f1)
        charuco = json.load(f2)
        
    # === 보드의 T_cam2target 변환행렬
    rvec = np.array(charuco["rvec_target2cam"]).reshape(3)
    tvec = np.array(charuco["tvec_target2cam"]).reshape(3, 1)
    R = o3d.geometry.get_rotation_matrix_from_axis_angle(rvec)
    T_cam2target = np.eye(4)
    T_cam2target[:3, :3] = R
    T_cam2target[:3, 3] = tvec.flatten()
    
    # === 로봇의 T_base2ee
    R_base2ee = np.array(pose["R_base2ee"])
    t_base2ee = np.array(pose["t_base2ee"]).reshape(3, 1)
    T_base2ee = np.eye(4)
    T_base2ee[:3, :3] = R_base2ee
    T_base2ee[:3, 3] = t_base2ee.flatten()
    
    # === 보드의 base 기준 위치 계산
    T_target2cam = np.linalg.inv(T_cam2target)
    T_base2target = T_base2ee @ np.eye(4)  # T_ee2target이 없으므로 단위행렬

    T_target_in_base.append(T_base2target @ T_cam2base @ T_target2cam)

len(T_target_in_base)

25

In [9]:
pcds = []
merged = o3d.geometry.PointCloud()
for rgb, depth, intr, T in zip(rgb_paths[:10], converted_files[:10], intr_paths[:10], T_target_in_base[:10]):
    
    pcd = rgbd_to_pcd(rgb, depth, intr)
    pcd_transformed = pcd.transform(T)
    merged += pcd_transformed

save_path = f"{MAIN_PATH}/camera_observed.ply"
o3d.io.write_point_cloud(save_path, merged)
print(f"✅ 평균 기준 PCD 저장 완료: {save_path}")

✅ 평균 기준 PCD 저장 완료: /home/ros/llm_robot/data/Calibration/Eye-to-Hand12/camera_observed.ply


In [10]:
pcds = []
merged = o3d.geometry.PointCloud()
for rgb, depth, intr, T in zip(rgb_paths[10:], converted_files[10:], intr_paths[10:], T_target_in_base[10:]):
    
    pcd = rgbd_to_pcd(rgb, depth, intr)
    pcd_transformed = pcd.transform(T)
    merged += pcd_transformed

save_path = f"{MAIN_PATH}/merged_target_pcd.ply"
o3d.io.write_point_cloud(save_path, merged)
print(f"✅ 평균 기준 PCD 저장 완료: {save_path}")

✅ 평균 기준 PCD 저장 완료: /home/ros/llm_robot/data/Calibration/Eye-to-Hand12/merged_target_pcd.ply


In [12]:
# === PCD 불러오기 ===
source_pcd = o3d.io.read_point_cloud(f"{MAIN_PATH}/camera_observed.ply")  # 카메라 좌표계
target_pcd = o3d.io.read_point_cloud(f"{MAIN_PATH}/merged_target_pcd.ply")  # base 기준 기준 모델

# === 초기 정렬 적용: 카메라 좌표계 → base 좌표계
z_mean = np.mean(np.asarray(source_pcd.points)[:, 2])
source_pcd.translate((0, 0, -z_mean))
source_pcd.transform(T_cam2base)

source_pcd = source_pcd.select_by_index(
    np.where(np.asarray(source_pcd.points)[:, 2] < 1.0)[0]
)

# === ICP 정합
threshold = 0.01  # 허용 거리 (m)
icp_result = o3d.pipelines.registration.registration_icp(
    source_pcd, target_pcd,
    threshold,
    np.eye(4),
    o3d.pipelines.registration.TransformationEstimationPointToPoint()
)

# === 보정된 행렬
T_icp = icp_result.transformation
T_cam2base_refined = T_icp @ T_cam2base

# === 최종 R, t 추출
R_final = T_cam2base_refined[:3, :3]
t_final = T_cam2base_refined[:3, 3].reshape(3, 1)

print("최종 R_cam2base:")
print(R_final)
print("최종 t_cam2base:")
print(t_final)

최종 R_cam2base:
[[ 0.99865734  0.03501759  0.03817444]
 [ 0.03408911 -0.99911334  0.02470781]
 [ 0.0390058  -0.0233733  -0.99896558]]
최종 t_cam2base:
[[ 0.0285736 ]
 [-0.12601269]
 [ 0.52731557]]


In [11]:
result = {
    "R_cam2base": R_final.tolist(),
    "t_cam2base": t_final.flatten().tolist()  # 1D 리스트로 저장
}

# 저장
with open(f"{MAIN_PATH}/cam2base_icp_point.json", "w") as f:
    json.dump(result, f, indent=2)

In [12]:
# === PCD 불러오기 ===
source_pcd = o3d.io.read_point_cloud(f"{MAIN_PATH}/camera_observed.ply")
target_pcd = o3d.io.read_point_cloud(f"{MAIN_PATH}/merged_target_pcd.ply")

# === source는 이미 transform 된 상태
source_pcd.transform(T_cam2base)

# ✅ target에도 노말 계산 추가!
target_pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.02, max_nn=30)
)

# === 초기 정렬 적용: 카메라 좌표계 → base 좌표계
source_pcd.transform(T_cam2base)

# === ICP 정합
threshold = 0.01  # 허용 거리 (m)
icp_result = o3d.pipelines.registration.registration_icp(
    source_pcd, target_pcd,
    threshold,
    np.eye(4),  # 이미 source_pcd는 transform된 상태이므로 초기값은 단위행렬
    o3d.pipelines.registration.TransformationEstimationPointToPlane()
)

# === 보정된 행렬
T_icp = icp_result.transformation
T_cam2base_refined = T_icp @ T_cam2base @ T_cam2base

# === 최종 R, t 추출
R_final = T_cam2base_refined[:3, :3]
t_final = T_cam2base_refined[:3, 3].reshape(3, 1)

print("최종 R_cam2base:")
print(R_final)
print("최종 t_cam2base:")
print(t_final)

KeyboardInterrupt: 

In [27]:
result = {
    "R_cam2base": R_final.tolist(),
    "t_cam2base": t_final.flatten().tolist()  # 1D 리스트로 저장
}

# 저장
with open(f"{MAIN_PATH}/cam2base2_icp_plane.json", "w") as f:
    json.dump(result, f, indent=2)

In [13]:
source_pcd.paint_uniform_color([1, 0, 0])  # 빨강
target_pcd.paint_uniform_color([0, 1, 0])  # 초록

o3d.visualization.draw_geometries([source_pcd, target_pcd])